In [5]:
#//*** Collects pre-sorted Comments
#//*** Cleans Comments
#//*** Runs tfidf
#//*** Perform feature reduction on tfidf using Truncated SVD, preservering around 98% of variance. Cuts the Columns down from 200k+ to 6000
#//*** Saves the Truncated SVD for modeling
import os
import sys
import time
from datetime import date
from datetime import datetime
import time
import json
import platform

import stoneburner
#//*** Custom Functions:
#//*** mr_clean_text(input_series)
#//*** tokenize_series(input_series)
#//*** remove_stop_words(input_series)

# //*** Imports and Load Data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#subreddits = ["wallstreetbets", "stocks", "wallstreetbetsOGs", "spacs", "investing", "pennystocks", "stockmarket", "options", "robinhoodpennystocks", "wallstreetbetsnew", "smallstreetbets"]
filepath = "./data/"
#filename_suffix = "_comments.csv.zip"
#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [6]:
#//*** Path to file aggregate
input_filename = "./data/processed_reddit_v4_1min.csv.zip"

output_filename = './ignore_folder/tsvd_model_ready_60min.csv.zip'

raw_df = pd.read_pickle(input_filename)
#//*** How many columns to pre
offset_target = "1"


raw_df

,time,open,high,low,close,volume,body,comment_count
0,2019-07-29 08:00:00,10.770750,10.866067,10.770750,10.866067,2492,Was my favorite HMD until INDX came out. short...,40
1,2019-07-29 09:00:00,10.675434,10.923256,10.675434,10.723092,2200,,0
2,2019-07-29 10:00:00,10.913725,10.923256,10.418080,10.429136,564758,,0
3,2019-07-29 11:00:00,10.422845,10.713560,10.418080,10.694497,247215,,0
4,2019-07-29 12:00:00,10.694497,10.713560,10.580117,10.646839,284815,,0
...,...,...,...,...,...,...,...,...
6863,2021-07-15 15:00:00,33.938900,35.430000,33.330000,35.329100,16629161,,0
6864,2021-07-15 16:00:00,35.330000,36.100000,34.670000,35.991100,19774603,,0
6865,2021-07-15 17:00:00,35.960000,39.200000,35.760000,39.170000,4218169,,0
6866,2021-07-15 18:00:00,39.170000,39.870000,38.250000,38.750000,3911023,,0


In [8]:
#//************************************************
#//*** Clean text body
#//************************************************

raw_df['token'] = stoneburner.remove_stop_words(stoneburner.tokenize_series(stoneburner.mr_clean_text(raw_df['body'],{"remove_empty":False})))
print("Done")

#//*** Detokenize the clean column as tfidf
raw_df['tfidf'] = raw_df['token'].apply(lambda x: ' '.join(x))


remove_empty False
Text Cleaning Time: 23.369927406311035
Tokenize Time: 26.14117121696472
Stop Words Time: 20.6454861164093
Done


In [9]:
#//*** Build tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

#loop_list.append(tfidf.fit_transform(input_df['tfidf']))
tfidf_matrix = []
tfidf_list = []
tfidf = TfidfVectorizer()

print("Starting tfidf....")
start_time = time.time()
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(raw_df['tfidf'])


print(f"Built: {round(time.time()-start_time,2)}")

print(tfidf_matrix)
print

Starting tfidf....
Built: 4.9
  (0, 14299)	0.03401961698924732
  (0, 74899)	0.04682019536046372
  (0, 81673)	0.030586264755460793
  (0, 48755)	0.015477205176876974
  (0, 54192)	0.02696689468332636
  (0, 109155)	0.024218678637912047
  (0, 95210)	0.017444858699409642
  (0, 56220)	0.027932558423931587
  (0, 105071)	0.09848800926392938
  (0, 60648)	0.06003137748849249
  (0, 65489)	0.02038803993539359
  (0, 112084)	0.05847955838699228
  (0, 107166)	0.03393851063884545
  (0, 76114)	0.030566550978695962
  (0, 58136)	0.02280262763934598
  (0, 101644)	0.013389877265936084
  (0, 36280)	0.027598467281734373
  (0, 75327)	0.029691281040632955
  (0, 90284)	0.06621325995138741
  (0, 54337)	0.020742196951851536
  (0, 20812)	0.03866277717354503
  (0, 104988)	0.07752359839186433
  (0, 61244)	0.025487585687380587
  (0, 76213)	0.02946300083732014
  (0, 79548)	0.03671942130676892
  :	:
  (6706, 79279)	0.03725580570711172
  (6706, 101607)	0.04767484929925102
  (6706, 97594)	0.03956442029169066
  (6706, 7113

<function print>

In [12]:
#//*** Remove Uneeded columns

#//*** Separate the time into a separate series
print("Removing uneeded columns...")
#//*** Remove uneeded columns
for col in ['body','token','tfidf']:
    if col in raw_df.columns:
        del raw_df[col]

raw_df

Removing uneeded columns...


,time,open,high,low,close,volume,comment_count
0,2019-07-29 08:00:00,10.770750,10.866067,10.770750,10.866067,2492,40
1,2019-07-29 09:00:00,10.675434,10.923256,10.675434,10.723092,2200,0
2,2019-07-29 10:00:00,10.913725,10.923256,10.418080,10.429136,564758,0
3,2019-07-29 11:00:00,10.422845,10.713560,10.418080,10.694497,247215,0
4,2019-07-29 12:00:00,10.694497,10.713560,10.580117,10.646839,284815,0
...,...,...,...,...,...,...,...
6863,2021-07-15 15:00:00,33.938900,35.430000,33.330000,35.329100,16629161,0
6864,2021-07-15 16:00:00,35.330000,36.100000,34.670000,35.991100,19774603,0
6865,2021-07-15 17:00:00,35.960000,39.200000,35.760000,39.170000,4218169,0
6866,2021-07-15 18:00:00,39.170000,39.870000,38.250000,38.750000,3911023,0


In [13]:
"""
#//*** Move this to modeling

input_filename = "./ignore_folder/processed_reddit_v4_60min.pkl.zip"
print("Building Target Offset Columns...")
#//*** Build the target variables and intervals of stock prices. This is a single value determined by target)offset

#//*** create a list of nan values of x length
nan_list = list(np.empty( offset_target )* np.nan )

#//*** Create target variable Price which is stocks + x columns in advance
#//*** Takes the closing price starting at x and gets the remainder, this generates the offset
#//*** nan_list fills the missing x values with nans
raw_df[f"close_{offset_target}"] = list(raw_df['close'][offset_target:]) + nan_list 
raw_df    
#start_time = time.time()
#print("Merging...")
##//*** Combine the tfidf sparse array with the stock values
#combined_df = pd.concat([raw_df,pd.DataFrame(tfidf_matrix.toarray())], axis=1)
#print (f"Done: {round(time.time()-start_time,2)}s")
"""
print()

Building Target Offset Columns...


,time,open,high,low,close,volume,comment_count,close_1
0,2019-07-29 08:00:00,10.770750,10.866067,10.770750,10.866067,2492,40,10.723092
1,2019-07-29 09:00:00,10.675434,10.923256,10.675434,10.723092,2200,0,10.429136
2,2019-07-29 10:00:00,10.913725,10.923256,10.418080,10.429136,564758,0,10.694497
3,2019-07-29 11:00:00,10.422845,10.713560,10.418080,10.694497,247215,0,10.646839
4,2019-07-29 12:00:00,10.694497,10.713560,10.580117,10.646839,284815,0,10.765984
...,...,...,...,...,...,...,...,...
6863,2021-07-15 15:00:00,33.938900,35.430000,33.330000,35.329100,16629161,0,35.991100
6864,2021-07-15 16:00:00,35.330000,36.100000,34.670000,35.991100,19774603,0,39.170000
6865,2021-07-15 17:00:00,35.960000,39.200000,35.760000,39.170000,4218169,0,38.750000
6866,2021-07-15 18:00:00,39.170000,39.870000,38.250000,38.750000,3911023,0,38.290000


In [43]:
#//*** Build TruncatedSVD

#from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import TruncatedSVD

start_time = time.time()
print(f"Begin Truncated SVD ")


#//*** Set the number of components to 6000. This is generating 98% variance capture
#//*** 60min data set took around 25minutes to build
tsvd = TruncatedSVD(6000)
tsvd_df = pd.DataFrame(tsvd.fit_transform(tfidf_matrix))
print(tsvd.explained_variance_ratio_.sum())

print (f"Truncated SVD Done: {round(time.time()-start_time,2)}s")

#//*** Write Truncated SVD to disk
tsvd_df.to_csv(output_filename, compression='zip', index=False)


Begin PCA 
0.9878555977289228
PCA Done: 1658.65s


In [21]:
#//****1k PCA - .45
#//****100 PCA = .13
pca.explained_variance_ratio_.sum()

0.1303087365492425

TruncatedSVD(n_components=6000)

In [49]:
import pickle 
#//*** Write the tsvd object to file, in case it's needed for additional analysis
outfile = open('./data/tsvd/truncatedSVD_60min.pkl.zip','wb')
pickle.dump(tsvd,outfile)
outfile.close()

In [ ]:
import sys
sys.getsizeof(combined_df)/1000000

In [ ]:
import time
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [ ]:
#//*** set the training columns to use everyting except time
predict_col = 'close_1'
train_cols = combined_df.columns[1:]
train_cols = ['close','volume']

x_train = combined_df[train_cols][:1000]
y_train = target_df[predict_col][:1000]
x_test = combined_df[train_cols][1001:1100]
y_test = target_df[predict_col][1001:1100]
x_train

In [ ]:
start_time = time.time()
print("Regressing")
regr = MLPRegressor(max_iter=500).fit(x_train,y_train)
#scores = cross_val_score(regr, x_train, y_train, cv=5)
#//*** Score the model
score = regr.score(x_train, y_train)
result = regr.predict(x_test)

mse = mean_squared_error(y_test, result)
r2 = r2_score(y_test,result)
#//*** Root Mean squared Error
rmse = sqrt(mse)
print(f"Complete: {round(time.time()-start_time,2)}" )

print(f"rmse: {rmse}")
print(f"r2: {r2}")

PCA Tutorial: https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

In [ ]:
print(f"Begin Train Cols ")
train_cols = combined_df.columns[1:]
predict_col = 'close_1'
print(f"Begin Slice ")

x_train = combined_df[train_cols][:1000]
y_train = target_df[predict_col][:1000]
x_test = combined_df[train_cols][1001:1013]
y_test = target_df[predict_col][1001:1013]

start_time = time.time()
print(f"Begin PCA ")

# %%
from sklearn.decomposition import PCA

scaler = MinMaxScaler()

pca = PCA(n_components=3)
#X = pca.fit_transform( scaler.fit_transform(x_train) ,y_train)
x_train = pd.DataFrame(pca.fit_transform( x_train ,y_train))
x_test = pd.DataFrame(pca.fit_transform( x_test ,y_train))

print("Explained Variance Ratio")
for x in range(len(pca.explained_variance_ratio_)):
    print(x, pca.explained_variance_ratio_[x])
print (f"PCA Done: {round(time.time()-start_time,2)}s")



In [ ]:
start_time = time.time()
print("Regressing")
regr = MLPRegressor(max_iter=1000).fit(x_train,y_train)
#scores = cross_val_score(regr, x_train, y_train, cv=5)
#//*** Score the model
score = regr.score(x_train, y_train)
result = regr.predict(x_test)

mse = mean_squared_error(y_test, result)
r2 = r2_score(y_test,result)
#//*** Root Mean squared Error
rmse = sqrt(mse)
print(f"Complete: {round(time.time()-start_time,2)}" )

print(f"rmse: {rmse}")
print(f"r2: {r2}")

In [ ]:
display_size = 40

fig,ax = plt.subplots()
plot_x = np.arange(len(y_test))
ax.plot(plot_x,y_test )
ax.scatter(plot_x,result,color='red' )


    #plt.legend(loc='upper right',bbox_to_anchor=(1.35, 1.2))
plt.show()

In [ ]:
for x in range(len(pca.explained_variance_ratio_)):
    print(x, pca.explained_variance_ratio_[x])

In [ ]:
y_train
